In [547]:
"""
скрипт для обработки данных мониторинга трудоустройства
"""


'\nскрипт для обработки данных мониторинга трудоустройства\n'

In [548]:
import pandas as pd
import numpy as np
import openpyxl
import warnings
import copy
import math

warnings.simplefilter(action='ignore', category=UserWarning)

In [549]:
def check_data(cell:str):
    """
    Метод для проверки значения ячейки
    :param cell: значение ячейки
    :return: число в формате int
    """
    if cell is np.nan:
        return 0
    if cell.isdigit():
        return int(cell)
    else:
        return 0


In [550]:
path_to_file ='data/Брит - 2022.xlsx'
path_to_end_folder = 'data/temp'
name_file = 'БРИТ'

In [551]:
df = pd.read_excel(path_to_file,skiprows=7,dtype=str)

In [552]:
high_level_dct = {}

In [553]:
df.columns = list(map(str,df.columns))

In [554]:
# Заполняем пока пропуски в 15 ячейке для каждой специальности
df['06'] = df['06'].fillna('15 ячейка')

In [555]:
# очищаем от нан и возможнных пустых пробелов
code_spec =[spec for spec in df['03'].unique() if spec is not np.nan]
code_spec =[spec for spec in code_spec if spec != ' ']

In [556]:
# Создаем список для строк
row_cat = [f'Строка {i}' for i in range(1,16)]
# Создаем список для колонок
column_cat = [f'Колонка {i}' for i in range(7,34)]

In [557]:
column_cat

['Колонка 7',
 'Колонка 8',
 'Колонка 9',
 'Колонка 10',
 'Колонка 11',
 'Колонка 12',
 'Колонка 13',
 'Колонка 14',
 'Колонка 15',
 'Колонка 16',
 'Колонка 17',
 'Колонка 18',
 'Колонка 19',
 'Колонка 20',
 'Колонка 21',
 'Колонка 22',
 'Колонка 23',
 'Колонка 24',
 'Колонка 25',
 'Колонка 26',
 'Колонка 27',
 'Колонка 28',
 'Колонка 29',
 'Колонка 30',
 'Колонка 31',
 'Колонка 32',
 'Колонка 33']

In [558]:
# Создаем словарь нижнего уровня содержащий в себе все данные для каждой специальности
spec_dict = {}
for row in row_cat:
    spec_dict[row] = {key:0 for key in column_cat}
# Изменяем последний ключ на строковый поскольку там будут хранится примечания
spec_dict['Колонка 33'] = ''

In [559]:
# Создаем словарь среднего уровня содержащй данные по всем специальностям
poo_dct = {key:copy.deepcopy(spec_dict) for key in code_spec}

In [560]:
id(poo_dct['13.01.10']['Строка 1'])

2243442616064

In [561]:
high_level_dct[name_file] = poo_dct

In [562]:
"""
В итоге получается такая структура
{БРИТ:{13.01.10:{Строка 1:{Колонка 1:0}}},ТСИГХ:{22.01.10:{Строка 1:{Колонка 1:0}}}}

"""

'\nВ итоге получается такая структура\n{БРИТ:{13.01.10:{Строка 1:{Колонка 1:0}}},ТСИГХ:{22.01.10:{Строка 1:{Колонка 1:0}}}}\n\n'

In [563]:
current_code = 'Ошибка проверьте правильность заполнения кодов специальностей' # чекбокс для проверки заполнения кода специальности

In [564]:
idx_row = 1 # счетчик обработанных строк

In [565]:
# Итерируемся по полученному датафрейму через itertuples
for row in df.itertuples():
    # если счетчик колонок больше 15 то уменьшаем его до единицы
    if idx_row >15:
        idx_row = 1
    
    #print(row)
    # Проверяем на незаполненные ячейки и ячейки заполненные пробелами
    if (row[3] is not np.nan) and (row[3] != ' '):
        # если значение ячейки отличается от текущего кода специальности то обновляем значение текущего кода
        if row[3] != current_code:
            current_code = row[3]
        data_row = row[7:34] # получаем срез с нужными данными     
        for idx_col,value in enumerate(data_row,start=1):
            #print(f'Строка {idx_row}')
            print(f'Колонка {idx_col}')
            print(value)
            if idx_col + 6 == 33:
                # сохраняем примечания в строке
                high_level_dct[name_file][current_code][f'Строка {idx_row}'][f'Колонка {idx_col + 6}'] = value
            else:                
                high_level_dct[name_file][current_code][f'Строка {idx_row}'][f'Колонка {idx_col + 6}'] += check_data(value)

    idx_row += 1
                
            


    
    
    

Колонка 1
21
Колонка 2
8
Колонка 3
7
Колонка 4
nan
Колонка 5
nan
Колонка 6
nan
Колонка 7
9
Колонка 8
3
Колонка 9
nan
Колонка 10
1
Колонка 11
nan
Колонка 12
nan
Колонка 13
nan
Колонка 14
nan
Колонка 15
nan
Колонка 16
nan
Колонка 17
nan
Колонка 18
nan
Колонка 19
nan
Колонка 20
nan
Колонка 21
nan
Колонка 22
nan
Колонка 23
nan
Колонка 24
nan
Колонка 25
nan
Колонка 26
nan
Колонка 27
пример примечания
Колонка 1
24
Колонка 2
2
Колонка 3
1
Колонка 4
nan
Колонка 5
nan
Колонка 6
nan
Колонка 7
19
Колонка 8
nan
Колонка 9
nan
Колонка 10
3
Колонка 11
nan
Колонка 12
nan
Колонка 13
nan
Колонка 14
nan
Колонка 15
nan
Колонка 16
nan
Колонка 17
nan
Колонка 18
nan
Колонка 19
nan
Колонка 20
nan
Колонка 21
nan
Колонка 22
nan
Колонка 23
nan
Колонка 24
nan
Колонка 25
nan
Колонка 26
nan
Колонка 27
раппар54
Колонка 1
nan
Колонка 2
nan
Колонка 3
nan
Колонка 4
nan
Колонка 5
nan
Колонка 6
nan
Колонка 7
nan
Колонка 8
nan
Колонка 9
nan
Колонка 10
nan
Колонка 11
nan
Колонка 12
nan
Колонка 13
nan
Колонка 14
nan
Колонка

In [566]:
high_level_dct[name_file]

{'43.01.06': {'Строка 1': {'Колонка 7': 21,
   'Колонка 8': 8,
   'Колонка 9': 7,
   'Колонка 10': 0,
   'Колонка 11': 0,
   'Колонка 12': 0,
   'Колонка 13': 9,
   'Колонка 14': 3,
   'Колонка 15': 0,
   'Колонка 16': 1,
   'Колонка 17': 0,
   'Колонка 18': 0,
   'Колонка 19': 0,
   'Колонка 20': 0,
   'Колонка 21': 0,
   'Колонка 22': 0,
   'Колонка 23': 0,
   'Колонка 24': 0,
   'Колонка 25': 0,
   'Колонка 26': 0,
   'Колонка 27': 0,
   'Колонка 28': 0,
   'Колонка 29': 0,
   'Колонка 30': 0,
   'Колонка 31': 0,
   'Колонка 32': 0,
   'Колонка 33': 'пример примечания'},
  'Строка 2': {'Колонка 7': 0,
   'Колонка 8': 0,
   'Колонка 9': 0,
   'Колонка 10': 0,
   'Колонка 11': 0,
   'Колонка 12': 0,
   'Колонка 13': 0,
   'Колонка 14': 0,
   'Колонка 15': 0,
   'Колонка 16': 0,
   'Колонка 17': 0,
   'Колонка 18': 0,
   'Колонка 19': 0,
   'Колонка 20': 0,
   'Колонка 21': 0,
   'Колонка 22': 0,
   'Колонка 23': 0,
   'Колонка 24': 0,
   'Колонка 25': 0,
   'Колонка 26': 0,
   'Колонк

In [567]:
len(high_level_dct[name_file])

17

In [570]:
high_level_dct[name_file]['43.02.06']['Строка 3']['Колонка 33']

nan